### CoreBx_island_stderr_refac - Interpolate the North Core Banks stderr maps onto rotated 1-m grid and save  as a .nc file.

This does the same interpolation that CoreBx_island_refac does, but for the std err maps.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # for read_csv
import subprocess

import xarray as xr
import rioxarray
# from dask.distributed import LocalCluster
from scipy import interpolate, signal
from CoreBx_funcs import * # for 
%matplotlib inline

In [2]:
# source directory for std err maps
src_dir = 'C:/crs/proj/2021_FloSupp_Release/clipfest/std_err/'

# destination dir for cropped std err maps
crop_dir = 'C:/crs/proj/2019_DorianOBX/Dorian_paper_analyses/cropped_std_err/'

# destination dir for rotated maps
rot_dir = 'C:/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/'

In [3]:
# These are the original file names before _crop was appended...that gets added on below
src_fnames = (\
#'20181006_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif',\
#'20181007_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif',\
#'20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif',\
'20190830_Ocracoke_Inlet_to_Ophelia_Inlet_Revised_dense_NAD83_UTM18_NAVD88m_1m_SErr_cog.tif',\
#'20190908_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif',\
'20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif',\
'20191011_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_4D_165_201910-202005_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20191126_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_4D_165_201910-202005_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20200208-9_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_4D_165_201910-202005_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20200508-9_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_4D_165_201910-202005_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20200802_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_FloSup_4D_165_202008-202104_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20200805-8_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_FloSup_4D_165_202008-202104_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20200928_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_FloSup_4D_165_202008-202104_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif',\
'20210430_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_FloSup_4D_165_202008-202104_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog.tif')

#### Generate the coordinates for the rotated island coord. system.

In [4]:
# Read in a dict to define the rotated coordinate system
r = yaml2dict('island_box.yml')
print(r)

# Make a grid 
xu,yu,xrot,yrot,xcoords,ycoords = make_grid(**r)

ny,nx = np.shape(xu)
print('Size of grid:',ny,nx)

{'name': 'ncorebx_refac', 'e0': 378490.0, 'n0': 3855740.0, 'xlen': 36500.0, 'ylen': 1500.0, 'dxdy': 1.0, 'theta': 42.0}
make_grid: Shape of xrot, yrot:  (1500, 36500) (1500, 36500)
corners x, corners y]
[[ 378490.03700711 3855740.70613772]
 [ 405614.07999221 3880163.30413921]
 [ 404611.05321328 3881277.2782326 ]
 [ 377487.01022818 3856854.68023111]
 [ 378490.03700711 3855740.70613772]]
Saving to ncorebx_refac.csv
Size of grid: 1500 36500


#### Read in all of the cropped std err files and interpolate onto rotated island grid

In [5]:
%%time
# optional median-filter smoothing of original maps
smooth = False
# kernal size...this should be an odd number >= dxy/0.1
ksize = 3

dslist=[]
for i, src_file in enumerate(src_fnames):
    crop_file = src_file.replace('.tif','_crop.tif')
    rot_file = crop_file.replace('.tif','_rot.tif')
    crop_path = crop_dir+crop_file
    rot_path = rot_file+crop_file
    print(crop_path, rot_path)
    iswarned = False

    # The "masked" option puts in NaNs, but takes longer.
    da = rioxarray.open_rasterio( crop_path, masked=True )

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    if(smooth):
        # smooth with 2D running median
        zs = signal.medfilt2d(z, kernel_size=ksize)
    else:
        zs = z

    f = interpolate.RegularGridInterpolator( (y, x), zs, method='nearest')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))
        
    # this is the fast iteration, which only works when all of the source points fall inside the target box
    try:
        zi=f((yu,xu))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    except:
        if(not iswarned):
            print("Warning: using slow iteration.")
            iswarned = True

        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

    #dar = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    dar = xr.DataArray(zi,dims=['Cross-shore','Alongshore'],coords={'Cross-shore': ycoords, 'Alongshore' :xcoords })

    dar = dar.chunk()
    dslist.append(dar)

dsa = xr.concat(dslist, dim='map')
fn = rot_dir+r['name']+'_stderr.nc'
dsa.to_netcdf(fn)
print('Saved to '+fn)

C:/crs/proj/2019_DorianOBX/Dorian_paper_analyses/cropped_std_err/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_Revised_dense_NAD83_UTM18_NAVD88m_1m_SErr_cog_crop.tif 20190830_Ocracoke_Inlet_to_Ophelia_Inlet_Revised_dense_NAD83_UTM18_NAVD88m_1m_SErr_cog_crop_rot.tif20190830_Ocracoke_Inlet_to_Ophelia_Inlet_Revised_dense_NAD83_UTM18_NAVD88m_1m_SErr_cog_crop.tif
(25542,) (28132,) (1, 25542, 28132)
(28132,) (25542,) (25542, 28132)
C:/crs/proj/2019_DorianOBX/Dorian_paper_analyses/cropped_std_err/20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog_crop.tif 20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog_crop_rot.tif20190912-13_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog_crop.tif
(25542,) (28132,) (1, 25542, 28132)
(28132,) (25542,) (25542, 28132)
C:/crs/proj/2019_DorianOBX/Dorian_paper_analyses/cropped_std_err/20191011_Ocracoke_Inlet_to_Cape_Lookout_RuPaReX2_FitAddl_4D_165_201910-202005_dense_NAD832011_UTM_18N_NAVD88m_1m_SErr_cog_cr